In [12]:
from model_pytorch import *
import os 
import pickle
from cfg import Config


In [17]:
config = Config()

In [4]:
n_samples = 2 * int(df['length'].sum()/0.1)

In [5]:
prob_dist = class_dist /class_dist.sum()

In [6]:
np.random.choice(class_dist.index,p=prob_dist)

'Violin_or_fiddle'

In [18]:
def check_data():
    if os.path.isfile(config.p_path):
        print ('Loading existing data from {} model'.format(config.mode))
        with open(config.p_path,'rb') as handle:
            tmp = pickle.load(handle)
            return tmp
    else:
        return None

In [8]:
def build_rand_feat():
    tmp = check_data()
    if tmp:
        return tmp.data[0], tmp.data[1]
    
    X = []
    y = []
    _min,_max = float('inf'),-float('inf')
    for _ in tqdm_notebook(range(n_samples)):
        rand_class = np.random.choice(class_dist.index,p=prob_dist)
        file = np.random.choice(df[df.label==rand_class].index)
        rate, wav = wavfile.read('clean/'+file)
        label = df.at[file,'label']
        rand_index = np.random.randint(0,wav.shape[0]-config.step)
        sample = wav[rand_index:rand_index+config.step]
        X_sample = mfcc(sample,rate,numcep=config.nfeat
                       ,nfilt=config.nfilt,nfft=config.nfft)
        
#         X_sample= logfbank(sample,rate,nfilt=config.nfilt,nfft=config.nfft).T
        
        
        _min = min(np.amin(X_sample),_min)
        _max = max(np.amax(X_sample),_max)
        X.append(X_sample)
        y.append(classes.index(label))
    config.min = _min
    config.max = _max
    X,y = np.array(X), np.array(y)
    X = (X - _min) / (_max-_min)
    if config.mode == 'conv':
        X = X.reshape(X.shape[0],X.shape[1],X.shape[2],1)
    elif config.mode == 'time':
        X= X.reshape(X.shape[0],X.shape[1],X.shape[2])
    y= to_categorical(y,num_classes=10)
    config.data = (X,y)
    
    with open(config.p_path,'wb') as handle:
        pickle.dump(config,handle,protocol=2)
        
    return X,y

In [30]:
rate, wav = wavfile.read('clean/00353774.wav')

In [31]:
wav.shape

(72320,)

In [32]:
 X_sample = mfcc(wav[:100],rate,numcep=config.nfeat
                       ,nfilt=config.nfilt,nfft=config.nfft)
        

In [33]:
X_sample.shape

(1, 13)

In [ ]:
# class Config:
#     def __init__(self,mode='conv',nfilt=26,nfeat=13,nfft=512,rate=16000):
#         self.mode=mode
#         self.nfilt = nfilt
#         self.nfeat = nfeat
#         self.nfft = nfft
#         self.rate =rate
#         self.step = int(rate/10)

https://www.pyimagesearch.com/2018/12/31/keras-conv2d-and-convolutional-layers/

<img src="https://www.pyimagesearch.com/wp-content/uploads/2018/12/keras_conv2d_num_filters.png">

In [ ]:
def get_conv_model():
    model = Sequential()
    model.add(Conv2D(16,(3,3),activation='relu',strides=(1,1),padding='same',input_shape=input_shape))
    model.add(Conv2D(32,(3,3),activation='relu',strides=(1,1),padding='same'))
    model.add(Conv2D(64,(3,3),activation='relu',strides=(1,1),padding='same'))
    model.add(Conv2D(128,(3,3),activation='relu',strides=(1,1),padding='same'))
    model.add(MaxPool2D((2,2)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(128,activation='relu'))
    model.add(Dense(64,activation='relu'))
    model.add(Dense(10,activation='softmax'))
    model.summary()
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
    return model

In [9]:
class ConvNet(torch.nn.Module):

    def __init__(self, num_classes):
        super(ConvNet, self).__init__()
        
      
        # 28x28x1 => 28x28x8
        self.conv_1 = torch.nn.Conv2d(in_channels=1,
                                      out_channels=8,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      padding=1) 
        # 28x28x8 => 14x14x8
        self.pool_1 = torch.nn.MaxPool2d(kernel_size=(2, 2),
                                         stride=(2, 2),
                                         padding=0) # (2(14-1) - 28 + 2) = 0                                       
        # 14x14x8 => 14x14x16
        self.conv_2 = torch.nn.Conv2d(in_channels=8,
                                      out_channels=16,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      padding=1) # (1(14-1) - 14 + 3) / 2 = 1                 
        # 14x14x16 => 7x7x16                             
        self.pool_2 = torch.nn.MaxPool2d(kernel_size=(2, 2),
                                         stride=(2, 2),
                                         padding=0) # (2(7-1) - 14 + 2) = 0

        self.linear_1 = torch.nn.Linear(7*7*16, num_classes)

        
        
    def forward(self, x):
        out = self.conv_1(x)
        out = F.relu(out)
        out = self.pool_1(out)

        out = self.conv_2(out)
        out = F.relu(out)
        out = self.pool_2(out)
        
        logits = self.linear_1(out.view(-1, 7*7*16))
        probas = F.softmax(logits, dim=1)
        return logits, probas

In [19]:
X,y = build_rand_feat()

Loading existing data from conv model


In [21]:
model = ConvNet(n_samples)

In [35]:
X.shape

(26410, 9, 13, 1)

In [23]:
model(torch.tensor(X))

RuntimeError: Given groups=1, weight of size 8 1 3 3, expected input[26410, 9, 13, 1] to have 1 channels, but got 9 channels instead

In [ ]:
config= Config(mode='conv')

In [ ]:
if config.mode == 'conv':
    X,y = build_rand_feat()
    y_flat = np.argmax(y,axis=1)
    input_shape = (X.shape[1],X.shape[2],1)
    model = get_conv_model()
if config.mode == 'time':
    X,y = build_rand_feat()
    y_flat = np.argmax(y,axis=1)
    input_shape = (X.shape[1],X.shape[2])
    model = get_recurrent_model()

In [ ]:
model = get_recurrent_model()

In [ ]:
class_weight = compute_class_weight('balanced',np.unique(y_flat),y_flat)

In [ ]:
checkpoint = ModelCheckpoint(config.model_path,monitor = 'val_acc',verbose =1
                             ,mode='max',save_best_only=True,save_weights_only=False,period=1 )

In [ ]:

model.fit(X,y,epochs=10,batch_size=32,class_weight=class_weight
         ,shuffle=True,validation_split=0.1,callbacks=[checkpoint]
         )

In [ ]:
model.save(config.model_path)